# Manage Qiskit Serverless compute and data resources

Qiskit Serverless allows you to manage compute and data across your Qiskit pattern, including CPUs, QPUs, and other compute accelerators.

## Parallel workflows

For classical tasks that can be parallelized, use the `@distribute_task` decorater to define compute requirements needed to perform a task. Start by recalling the original `transpile_parallel.py` example:

In [ ]:
# /source_files/transpile_remote.py

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_serverless import distribute_task

@distribute_task(target={"cpu": 1})
def transpile_remote(circuit, optimization_level, backend):
    """Transpiles an abstract circuit into an ISA circuit for a given backend."""
    pass_manager = generate_preset_pass_manager(
        optimization_level=optimization_level,
		backend=backend
    )
    isa_circuit = pass_manager.run(circuit)
    return isa_circuit

In this example, you decorated the `transpile_remote()` method with `@distribute_task(target={"cpu": 1})`. When run, this creates an asynchronous parallel workers task with a single CPU core, and returns with a reference to track the worker. To fetch the result, pass the reference to the `get()` method:

In [ ]:
from qiskit_serverless import get

transpile_worker_reference = transpile_remote(circuit, optimization_level, backend)
result = get(transpile_worker_reference)

You can also create and run multiple parallel tasks as follows:

In [ ]:
transpile_worker_references = [
    transpile_remote(circuit, optimization_level, backend)
    for circuit in circuit_list
]

results = get(transpile_worker_references)

### Explore different task configurations

You can flexibly allocate CPU, GPU, and memory for your tasks via `@distribute_task()`. For Qiskit Serverless on IBM Quantum Platform, each program is equipped with 16 CPU cores and 32 GB RAM, which can be allocated dynamically as needed. 

CPU cores can be allocated as full CPU cores, or even fractional allocations, as shown in the following.

Memory is allocated as number of bytes. Recall that there are 1024 bytes in a kilobyte, 1024 kilobytes in a megabyte, and 1024 megabytes in a gigabyte. To allocate 2 GB of memory for your worker, you need to allocate `"mem": 2 * 1024 * 1024 * 1024` 

In [ ]:
@distribute_task(target={
    "cpu": 16,
    "mem": 32 * 1024 * 1024 * 1024
})
def transpile_remote(circuit, optimization_level, backend):
    return None

## Automatic QPU selection

This example demonstrates how to use `IBMQPUSelector` to automate the process of selecting which qubits to use from a set of available QPUs. Instead of manually selecting a QPU, Qiskit Serverless automatically allocates a QPU according to desired criteria. `IBMQPUSelector`s are imported from server-side `qiskit_serverless_tools.selectors` modules, and must be invoked from your uploaded program.

Here, `IBMLeastNoisyQPUSelector` finds the QPU that yields the least-noisy qubit subgraph for the input circuit, from among the QPUs available to you through your IBM Quantum account. 

For each `IBMQPUSelector`, the context is set in the constructor. All `IBMQPUSelectors` require Qiskit Runtime credentials. The `IBMLeastNoisyQPUSelector` requires a circuit and transpile options specifying how the circuit should be optimized for each QPU, to determine the most optimal QPU and qubit layout. 

All `IBMQPUSelectors` implement a `get_backend` method, which retrieves the optimal QPU with respect to the given context. The `get_backend` method also allows for additional filtering of the QPUs. It is implemented using the same interface as the [QiskitRuntimeService.backends method](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#backends).

In [ ]:
# source_files/transpile_parallel.py
 
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit.random import random_circuit
from qiskit_serverless_tools.selectors import IBMLeastNoisyQPUSelector

service = QiskitRuntimeService(channel='ibm_quantum', token=API_TOKEN)
 
abstract_circuit = random_circuit(
    num_qubits=5, depth=4, measure=True
)

selector = IBMLeastNoisyQPUSelector(
    service, circuit=abstract_circuit, transpile_options={"optimization_level": 3}
)
backend = selector.get_backend(min_num_qubits=127)
target_circuit = selector.optimized_circuit


You can also use the `IBMLeastBusyQPUSelector` to find a QPU that can support the circuit width but with the shortest queue.

In [ ]:
# source_files/transpile_parallel.py

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_serverless_tools.selectors import IBMLeastBusyQPUSelector

backend = IBMLeastBusyQPUSelector(service).get_backend(min_num_qubits=127)
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
target_circuit = pm.run(abstract_circuit)

## Manage data across your program

Qiskit Serverless allows you to manage files in the `/data` directory across all your programs. This includes several limitations:

- Only `tar` and `h5` files are supported today
- This is only a flat `/data` storage, and cannot have `/data/folder/` subdirectories

The following shows how to upload files:

In [5]:
from qiskit_serverless import IBMServerlessClient

# Authenticate to the remote cluster and submit the pattern for remote execution
serverless = IBMServerlessClient(
    token=API_TOKEN
)

In [8]:
import tarfile

# Create a tar
filename = "transpile_demo.tar"
file = tarfile.open(filename,"w")
file.add("source_files/transpile_remote.py")
file.close()

# Upload the tar to Serverless data directory
serverless.file_upload(filename)

'{"message":"/usr/src/app/media/5f37582aa306c50013fac285/transpile_demo.tar"}'

Next, we can list all the files in our `data` directory. This data is accessible to all programs.

In [9]:
serverless.files()

['transpile_demo.tar']

This can be used from a program by using `file_download()` to download the file to the program environment, and uncompressing the `tar`.

In [17]:
# transpile_parallel.py

import tarfile

files = serverless.files()
demo_file = files[0]
downloaded_tar = serverless.file_download(demo_file)

print(downloaded_tar)
with tarfile.open(downloaded_tar, 'r') as tar:
    tar.extractall()

100%|██████████| 10.2k/10.2k [00:00<00:00, 10.0MiB/s]

downloaded_3d3c4bf6_transpile_demo.tar


At this point, your program can interact with the files, as you would a local experiment. `file_upload()` , `file_download()`, and `file_delete()` can be called from your local experiment, or your uploaded program, enabling consistent and flexible data management.